# 📰 News Recommendation System Part 2 - Multi-Channel Recall

## 📘 Project Introduction
This project explores user behavior prediction in a news recommendation scenario. The goal is to build a model that can predict a user's future click behavior based on their historical browsing and clicking behavior data, specifically the last news article they clicked on.

The setting is inspired by a real-world news app, where delivering timely, relevant content is essential for user engagement. This project aims to simulate a practical recommender system, combining business intuition with machine learning techniques to address a realistic problem in the content recommendation space.

## 📊 Data Overview
The dataset contains user interaction data from a large-scale news platform, including:
- 300,000 users
- ~3 million clicks
- 360,000+ unique news articles; each news article is represented by a pre-trained embedding vector, capturing semantic relationships between articles.

We extracted click log data from 200,000 users as the training set, 50,000 users as test set A, and 50,000 users as test set B.

## 📄 Data Tables

- train_click_log.csv: Training set user click logs
- testA_click_log.csv: Test set user click logs
- articles.csv: News article information data table
- articles_emb.csv: Embedding vector representation of news articles

|        **Field**        |         **Description**          |
| :---------------------: | :------------------------------: |
|         user_id         |              User ID             |
|    click_article_id     |            Clicked article ID    |
|     click_timestamp     |            Click timestamp        |
|    click_environment    |             Click environment     |
|    click_deviceGroup    |            Click device group     |
|        click_os         |           Click operating system  |
|      click_country      |             Click city            |
|      click_region       |             Click region          |
|   click_referrer_type   |           Click source type       |
|       article_id        | Article ID, corresponding to click_article_id |
|       category_id       |            Article type ID        |
|      created_at_ts      |          Article creation timestamp |
|       words_count       |             Article word count     |
| emb_1,emb_2,...,emb_249 |      Article embedding vector representation |

## 📏 Evaluation Metrics
The final recommendation for each user will include five recommended articles, sorted by click probability.

For example, for user1, our recommendation would be:
> user1, article1, article2, article3, article4, article5.

There is only one correct answer for each user's last clicked article, so we check if any of the recommended five articles match the actual answer. We will use **mean reciprocal rank** as the evaluation metric. The formula is as follows:
$$
score(user) = \sum_{k=1}^5 \frac{s(user, k)}{k}
$$

If article1 is the actual article clicked by the user, then s(user1, 1) = 1, and s(user1, 2-4) are all 0. If article2 is the article clicked by the user, then s(user, 2) = 1/2, and s(user, 1, 3, 4, 5) are all 0. Thus, score(user) = the reciprocal of the rank at which the match occurs. If there are no matches, score(user1) = 0. This is reasonable because we want hits to be as high-ranking as possible, which yields a higher score.

## 💡 Project Understanding
The goal of this project is to **predict the last news article a user clicked, based on their historical browsing data**. Unlike traditional structured prediction problems, this is more aligned with real-world recommendation systems, using raw user click logs rather than neatly labeled data.

To approach this, I framed the task as a **supervised learning** problem by transforming user-article interactions into "features + labels" training data. The core idea is to predict the likelihood of a user clicking a given article, turning this into a click-through rate (CTR) prediction task. This reframing allows for the use of **classification models**—starting with simple baselines like logistic regression and moving toward deep learning approaches.

Now, we have converted this problem into a classification problem, where the classification label is whether the user will click on a particular article. The features of the classification problem will include the user and the article. We need to train a classification model to predict the probability of a particular user clicking on a specific article. This raises several additional questions:
- How to create training and testing datasets?
- What specific features can we leverage?
- What models can we attempt?
- With 360,000 articles and over 200,000 users, what strategies do we have to reduce the problem's scale? How do we make the final predictions?

**For the second part, we will use a multi-channel recall strategy to create a set of candidate news articles for the recommendation system. This approach is also designed to solve the cold-start problem for new users.**

Multi-channel recall is a powerful strategy that uses several different methods to generate a diverse pool of relevant content.
- **Multiple Strategies**: The system uses a variety of simple, fast methods to find potential recommendations. Each method, or "channel," looks at the data from a different angle. For example, one channel might find articles similar to what the user has clicked on before, while another might find the most popular articles overall.
- **Balancing Speed and Quality**: These channels are designed to be fast, so the system can quickly find a large number of candidates. This keeps the recommendation process from being slow. By using many diverse channels, the system ensures a **high recall rate**, meaning it's very likely to find the most relevant items, which then perform well in the final ranking stage.
- **Efficiency**: Because each channel operates independently, they can all run at the same time using multi-threading. This parallel processing significantly speeds up the entire process.

In [ ]:
# Install the following packages if not installed
!pip install faiss-cpu
!pip install deepmatch
!pip install deepctr
!pip install tensorflow

1. **Faiss** (faiss-cpu): a library for efficient similarity search and clustering of dense vectors, allowing you to quickly find the **"nearest neighbors"** of a given vector. The 'faiss-cpu' package is a CPU-based version of Faiss

- Key Feature: Faiss provides a variety of fast search algorithms that are optimized for large-scale nearest neighbor search tasks in high-dimensional vector spaces.

- Best For: When you need to find similar items from massive datasets, such as searching for images, articles, or products that are similar to a user's preferences. It's a go-to for building the recall stage of a recommendation system.

2. **DeepMatch**: a library for building deep learning-based matching models in recommendation systems. It's designed to solve the **"matching"** problem—finding relevant items from a large pool that a user might be interested in.

- Key Feature: The library provides ready-to-use deep learning models for matching users to items, often used for the recall stage of recommendation systems in e-commerce, content platforms, and social networks.

- Best For: Quickly building and experimenting with personalized recommendation systems. It includes tools for data preprocessing and model evaluation, making it a powerful choice for rapid development.

3. **DeepCTR**: a Python library for **click-through rate (CTR) prediction** using deep learning models. CTR prediction is the task of estimating the probability that a user will click on an item.

- Key Feature: It offers a wide range of pre-built deep learning-based models and tools to build recommendation systems, ad click prediction models, and other CTR-related tasks.

- Best For: Building the ranking stage of a recommendation system, where you need a sophisticated model to predict which items a user is most likely to click on from a candidate set, and perform model evaluation and comparison.

## 1. Import Packages

In [1]:
# Libraries for Data Processing and Analysis
import pandas as pd  # For data processing and analysis, providing powerful data structures and tools.
import numpy as np  # For scientific computing, supporting multi-dimensional arrays and matrix operations.

# Libraries for Data Structures and Mathematical Operations
from collections import defaultdict, Counter  # Creates dictionaries with default values, automatically assigning them to non-existent keys.
import collections  # Provides additional data structures like counters and ordered dictionaries.
import math  # Provides mathematical functions, including trigonometric and logarithmic operations.

# Libraries for Date and Time Handling
from datetime import datetime  # For handling dates and times, including creation, formatting, and time calculations.
import time  # Import the time module for handling time-related operations.

# Miscellaneous Libraries 
import gc  # Import the garbage collection module for releasing memory space.
from operator import itemgetter  # Import the itemgetter function from the operator module for retrieving elements based on an index or key.
import os  # For interacting with the operating system, such as file operations and path handling.
from tqdm import tqdm  # For displaying progress bars, useful for tracking progress when handling large datasets.
import warnings
warnings.filterwarnings('ignore')  # Controls the display of warnings, ignoring all warnings here.
import pickle  # For serializing and deserializing Python objects, allowing saving to or loading from files.

# Libraries for Similarity Search and Clustering
import faiss  # For efficient similarity search and clustering algorithms.

# Libraries for Random Number Generation and Feature Scaling
import random  # Generates pseudo-random numbers.
from sklearn.preprocessing import MinMaxScaler  # Scales features to a specific range.
from sklearn.preprocessing import LabelEncoder  # Import the LabelEncoder class for label encoding.

# Libraries for deepctr, tensorflow, deepmatch
from deepctr.feature_column import SparseFeat, VarLenSparseFeat  # From the DeepCTR library, used to define sparse and variable-length sparse features.
import tensorflow as tf
from tensorflow.python.keras import backend as K  # From TensorFlow's Keras, provides low-level operations for building and training deep learning models.
from tensorflow.python.keras.models import Model  # For building neural network models, defining their structure and training process.
from tensorflow.keras.preprocessing.sequence import pad_sequences  # For sequence padding, ensuring sequences have uniform length.
from deepmatch.models import *  # Imports all models from the DeepMatch library, used for recommendation systems and related tasks.
from deepmatch.utils import sampledsoftmaxloss, NegativeSampler  # From the DeepMatch library, a custom loss function for specific deep learning tasks.

In [ ]:
# If using Google Colab, use this cell to load data
from google.colab import drive

# Connect to Google Drive
drive.mount('/content/drive')

# Define file paths
data_path = '/content/drive/MyDrive/Datasets/news-rec-sys/'
save_path = '/content/drive/MyDrive/Datasets/news-rec-sys/temp_results/'

In [2]:
# If using a local machine
data_path = './data/' 
save_path = './data/temp_results/' # save temperary result

In a recommendation system project, we can use different data loading modes to handle data efficiently and effectively. This approach saves time and resources while allowing for thorough testing and validation.

1. **Debug Mode** (Fast Development):
This mode is for rapidly building and testing a baseline model. Because recommendation datasets are often massive, trying to work with the full data from the start is inefficient. Instead, we use a small, random sample of the training data (train_click_log_sample). This allows us to quickly confirm that our code is working correctly before we scale up to larger datasets.

2. **Offline Validation Mode** (Model Selection & Tuning):
This is where we evaluate and fine-tune our models before putting them into production. We load the full training dataset (train_click_log) and then split it into a smaller training set and a validation set. The training set is used to train the model, while the validation set helps us test different model architectures and adjust hyperparameters to find the best configuration.

3. **Online Mode** (Final Prediction):
This final mode is for making predictions on the unseen test data. After developing a working baseline and selecting an optimal model, we combine all available data (both train_click_log and test_click_log) to create a comprehensive "full dataset." The model is then trained on this complete dataset to make the final predictions for the test set.

These three modes provide a structured workflow that is both practical and efficient for developing high-performance recommendation systems.

## 2. Functions for loading and preprocessing training and testing sets

In [3]:
# A standard function for memory optimization
def reduce_mem(df):
    starttime = time.time()  # Record the start time of the function
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']  # List of numeric data types
    start_mem = df.memory_usage().sum / 1024**2  # Calculate the memory usage of the DataFrame (in Mb)

    # Iterate through each column of the DataFrame
    for col in df.columns:
        col_types = df[col].dtypes  # Get the data type of the column
        if col_type in numerics:
            c_min = df[col].min()  # Get the minimum value in the column
            c_max = df[col].max()  # Get the maximum value in the column

            # Check if there are missing values in the minimum and maximum values
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue

            # Choose the appropriate data type conversion based on the data type's range
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
                    
    end_mem = df.memory_usage().sum / 1024**2  # Calculate the memory usage of the DataFrame after conversion (in Mb)
    print('-- Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction), time spend:{:2.2f} min'.format(end_mem,
                                                                                                  100*(start_mem-end_mem)/start_mem,
                                                                                                  (time.time()-starttime)/60))
    return df

In [4]:
# Debug mode: Sample a portion of data from the training set for code debugging
def get_all_click_sample(data_path, sample_nums=20000):
    """
    Sample a portion of the training data for debugging
    data_path: Path where the original data is stored
    sample_nums: Number of samples to extract (sample a smaller number of users due to memory limitations)
    """
    file_path = os.path.join(data_path, 'train_click_log.csv')  # use os.path.join to construct file path
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}!")
        
    all_click = pd.read_csv(file_path)
    all_user_ids = all_click['user_id'].unique()  # Get unique identifiers for all users
    
    # Randomly select a specified number of users from all users as sampled users
    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False)
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]  # Retain click data for sampled users
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))  # Remove duplicates
    return all_click


# Load all click data, divided into online and offline modes.
# If the goal is to validate model or feature effectiveness offline, we can use the training set only.
# If the goal is to train an online inference model, the test dataset should be merged with the full dataset.
def get_all_click_df(data_path, offline=True):
    if offline:
        file_path = os.path.join(data_path, 'train_click_log.csv') # use os.path.join to construct file path
        if not file_path:
            raise FileNotFoundError(f"File not found: {file_path}!")
        all_click = pd.read_csv(file_path)
    else:
        trn_click_path = os.path.join(data_path, 'train_click_log.csv') # use os.path.join to construct file path
        tst_click_path = os.path.join(data_path, 'testA_click_log.csv') # use os.path.join to construct file path

        if not os.path.exists(trn_click_path):
            raise FileNotFoundError(f"File not found: {trn_click_path}!")
        if not os.path.exists(tst_click_path):
            raise FileNotFoundError(f"File not found: {tst_click_path}!")
            
        trn_click = pd.read_csv(trn_click_path)
        tst_click = pd.read_csv(tst_click_path)
        all_click = pd.concat([trn_click, tst_click])
        
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))  # Remove duplicates
    return all_click

In [5]:
# Load basic attributes of articles
def get_item_info_df(data_path):
    file_path = os.path.join(data_path, 'articles.csv') # use os.path.join to construct file path
    if not file_path:
        raise FileNotFoundError(f"File not found: {file_path}!")
        
    item_info_df = pd.read_csv(file_path)
    
    # Rename article_id to click_article_id to merge with click_article_id in the training set
    item_info_df = item_info_df.rename(columns={'article_id': 'click_article_id'})
    return item_info_df

In [6]:
# Load article embedding data
# Read the CSV file containing article embeddings and convert them into an embedding dictionary
def get_item_emb_dict(data_path):
    file_path = os.path.join(data_path, 'articles_emb.csv') # use os.path.join to construct file path
    if not file_path:
        raise FileNotFoundError(f"File not found: {file_path}!")
        
    item_emb_df = pd.read_csv(file_path)

    # Select columns containing 'emb' to retrieve embedding vectors
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    # Convert the embedding columns into a NumPy array stored in contiguous memory
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols])
    # Normalize the embedding vectors to have a norm of 1 for comparability
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)

    # Create a dictionary mapping article IDs to their corresponding normalized embeddings
    item_emb_dict = dict(zip(item_emb_df['article_id'], item_emb_np))
    # Save the embedding dictionary to a file using pickle
    pickle.dump(item_emb_dict, open(save_path + 'item_content_emb.pkl', 'wb'))

    return item_emb_dict

In [7]:
# Load Samples
#all_click_df = get_all_click_sample(data_path, sample_nums=2000)

# Load the full training set
all_click_df = get_all_click_df(data_path, offline=True)

# Normalize the timestamps to calculate weights for association rules.
max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
all_click_df['click_timestamp'] = all_click_df[['click_timestamp']].apply(max_min_scaler)

In [8]:
# Load article info
item_info_df = get_item_info_df(data_path)

# Load embedding dictionary
item_emb_dict = get_item_emb_dict(data_path)

## 3. Utility functions

### 3.1 Get user-article-time dictionary
This function will be used in user-based collaborative filtering with association rules

In [9]:
# Retrieve the dictionary of users (key) and sequence of clicked by users based on click timestamps (value).   
# {user1: [(item1, time1), (item2, time2)..]...}
def get_user_item_time(click_df):
    """
    Create a dictionary where the key is the User ID, and the value is a list of ariticle id along with the timestamps clicked by the user.
    :param click_df: DataFrame containing user click information
    :return: Dictionary mapping User IDs to lists of article-click timestamp pairs
    """
    def make_item_time_pair(df):
        return list(zip(df['click_article_id'], df['click_timestamp']))
        
    click_df = click_df.sort_values('click_timestamp')
    user_item_time_df = click_df.groupby('user_id')[['click_article_id', 'click_timestamp']]\
                                .apply(lambda x: make_item_time_pair(x))\
                                .reset_index().rename(columns={0: 'item_time_list'})
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))

    return user_item_time_dict

### 3.2 Get article-user-time dictionary
This function will be used in item-based collaborative filtering with association rules.

In [10]:
# Retrieve the dictionary of items (key) and sequence of users based on click timestamps (value).   
# {item1: [(user1, time1), (user2, time2)...]...}
def get_item_user_time_dict(click_df):
    """
    Create a dictionary where the key is the item ID, and the value is a list of users who clicked the item along with the timestamps.
    :param click_df: DataFrame containing user click information
    :return: Dictionary mapping item IDs to lists of user-click timestamp pairs
    """
    def make_user_time_pair(df):
        return list(zip(df['user_id'], df['click_timestamp']))
        
    click_df = click_df.sort_values('click_timestamp')
    item_user_time_df = click_df.groupby('click_article_id')[['user_id', 'click_timestamp']]\
                                .apply(lambda x: make_user_time_pair(x))\
                                .reset_index().rename(columns={0: 'user_time_list'})
    item_user_time_dict = dict(zip(item_user_time_df['click_article_id'], item_user_time_df['user_time_list']))
    
    return item_user_time_dict

### 3.3 Get historical and last clicks dataframes 
This will be used in evaluating recall results, feature engineering, and creating labels for converting into a supervised learning test set.

In [11]:
# Get historical and last clicks from the current data
def get_hist_and_last_click(click_df):
    click_df = click_df.sort_values(by=['user_id', 'click_timestamp'])
    click_last_df = click_df.groupby('user_id').tail(1)

    # If the user has only one click record (len(user_df) == 1),
    # the hist_func will return the entire click record for that user.
    # Otherwise, it will return all clicks except the last one.
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = click_df.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

### 3.4 Get Article Attribute Features (dictionaries)

In [12]:
#  Retrieve the basic attributes corresponding to article IDs and save them as a dictionary for easy use during the recall and cold start phases.
def get_item_info_dict(item_info_df):
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    item_info_df['created_at_ts'] = item_info_df[['created_at_ts']].apply(max_min_scaler)

    item_type_dict = dict(zip(item_info_df['click_article_id'], item_info_df['category_id']))
    item_words_dict = dict(zip(item_info_df['click_article_id'], item_info_df['words_count']))
    item_created_time_dict = dict(zip(item_info_df['click_article_id'], item_info_df['created_at_ts']))

    return item_type_dict, item_words_dict, item_created_time_dict

### 3.5 Get Information of Articles Clicked in User's History

In [13]:
def get_user_hist_item_info_dict(click_df):
    """
    This function retrieves various information from the user's historical clicks and returns the following dictionaries:
    1. A dictionary of the set of article categories clicked by the user.
    2. A dictionary of the set of article IDs clicked by the user.
    3. A dictionary of the average word count of articles clicked by the user.
    4. A dictionary of the normalized creation time of the last article clicked by the user.
    """
    
    # Retrieve a dictionary of the set of article categories clicked by each user (user_id).
    user_hist_item_types = click_df.groupby('user_id')['category_id'].agg(set).reset_index()
    user_hist_item_types_dict = dict(zip(user_hist_item_types['user_id'], user_hist_item_types['category_id']))

    # Retrieve a dictionary of the set of article IDs clicked by each user (user_id).
    user_hist_item_ids_dict = click_df.groupby('user_id')['click_article_id'].agg(set).reset_index()
    user_hist_item_ids_dict = dict(zip(user_hist_item_ids_dict['user_id'], user_hist_item_ids_dict['click_article_id']))

    # Retrieve a dictionary of the average word count of articles clicked by each user (user_id).
    user_hist_item_words = click_df.groupby('user_id')['words_count'].agg('mean').reset_index()
    user_hist_item_words_dict = dict(zip(user_hist_item_words['user_id'], user_hist_item_words['words_count']))

    # Retrieve the creation time of the last article clicked by each user (user_id).
    click_df_ = click_df.sort_values('click_timestamp')
    user_last_item_created_time = click_df_.groupby('user_id')['created_at_ts'].apply(lambda x: x.iloc[-1]).reset_index()

    # Normalize created_at_ts
    max_min_scaler = lambda x : (x-np.min(x))/(np.max(x)-np.min(x))
    user_last_item_created_time['created_at_ts'] = user_last_item_created_time[['created_at_ts']].apply(max_min_scaler)

    user_last_item_created_time_dict = dict(zip(user_last_item_created_time['user_id'], \
                                                user_last_item_created_time['created_at_ts']))

    return user_hist_item_types_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict

The importance of these four dictionaries:
- **User Interest Categories**: By collecting and analyzing the categories of articles a user clicks on, we can build a profile of their interests. This set of categories is a powerful tool for making more personalized and accurate recommendations.

- **User Click History**: Keeping a record of every article a user has clicked on allows us to understand their historical behavior. This click sequence is essential for building sophisticated sequential models, which can predict future interests based on past actions.

- **Reading Habits**: Calculating the average word count of the articles a user reads helps us understand their reading preferences. This feature can be used to recommend articles that are a suitable length, improving the user's experience.

- **Recency and Timeliness**: Tracking the creation time of the most recent article a user clicked on is crucial. The latest clicks are often the most accurate reflection of a user's current interests, so we can use this information to prioritize fresh recommendations.

### 3.6 Retrieve the Top-K Most Clicked Articles

In [14]:
# Retrieve the Most Clicked Recent Articles
def get_item_topk_click(click_df, k):
    topk_click = click_df['click_article_id'].value_counts().index[:k]
    return topk_click

## 4. Define a multi-channel recall dictionary

In [15]:
# Retrieve Article Attribute Information and Store as a Dictionary for Easy Lookup
item_type_dict, item_words_dict, item_created_time_dict = get_item_info_dict(item_info_df)

# Extract user history and last clicks
trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)

In [16]:
# Define a Multi-Channel Recall Dictionary to Store Results from Different Recall Strategies
user_multi_recall_dict =  {'itemcf_sim_itemcf_recall': {},
                           'embedding_sim_item_recall': {},
                           'youtubednn_recall': {},
                           'youtubednn_usercf_recall': {},
                           'cold_start_recall': {}}

- `'itemcf_sim_itemcf_recall': {}`: This dictionary is for storing results from item-based collaborative filtering. It will hold the candidate articles retrieved by this algorithm, which finds items similar to a user's past clicks.
- `'embedding_sim_item_recall': {}`: This dictionary is for results from an embedding similarity algorithm. It will store the candidate articles found by comparing the vector embeddings of items to find those that are similar.
- `'cold_start_recall': {}`: This is a dedicated dictionary for handling the cold-start problem. It will contain the recall results for new users who have little to no historical data.
-  `'youtubednn_recall': {}`: This dictionary is for storing results from a YouTube DNN (Deep Neural Network) model. This model learns to represent users and items as vectors (embeddings) and then uses a fast similarity search to find the most relevant items for a given user. This dictionary will hold the candidate articles retrieved by this model.
-  `'youtubednn_usercf_recall': {}`: This is a more specific and advanced recall strategy that combines the power of the YouTube DNN with user-based collaborative filtering. This approach first uses a DNN model to find a user's embedding, then find other users with similar embeddings, and finally recommend items that those similar users have liked. This dictionary will hold the candidate articles generated through this hybrid method.

## 5. Recall Evaluation Function  
We need to evaluate our recall process and adjust the current recall method or parameters to achieve better recall performance. Since the recall results determine the upper limit of the final ranking, the following function provides a method for recall evaluation.

In [17]:
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=50):
    """
    This function evaluates the hit rate for the top 10, 20, 30, 40, and 50 recalled articles.
    """

    # Convert the user's last click records into a dictionary with user_id as the key and click_article_id as the value
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['click_article_id']))

    # Get the number of users in the recall dictionary
    user_num = len(user_recall_items_dict)

    # Evaluate the hit rate for the top 10, 20, 30, 40, and 50 recalled articles
    for k in range(10, topk + 1, 10):
        hit_num = 0  # Record the number of hits

        # Iterate through each user and their list of recalled articles
        for user, item_list in user_recall_items_dict.items():
            # Get the top-k recall results (article IDs)
            tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]

            # If the user's last clicked article is in the top-k recall results, count it as a hit
            if last_click_item_dict[user] in set(tmp_recall_items):
                hit_num += 1

        # Calculate the hit rate: hit rate = number of hits / total number of users
        hit_rate = round(hit_num * 1.0 / user_num, 5)

        # Output the result
        print('topk:', k, ' | hit_num:', hit_num, ' | hit_rate:', hit_rate, ' | user_num:', user_num)

## 6. Calculate the Similarity Matrix  
This section focuses on obtaining the similarity matrix through collaborative filtering and vector search. The similarity matrix is divided into `user2user` and `item2item`. 

### 6.1 Item Collaborative Filtering - item2item similarity 
We first obtain the `item2item` similarity matrix based on item-based collaborative filtering (itemCF). When calculating the item-to-item similarity matrix using association rules, the similarity of articles also takes into account:

- **Time weight of user clicks**: The recency of a user's click.

- **Sequence weight of user clicks**: The order in which a user clicked on articles.

- **Creation time weight of articles**: The recency of the article's publication.

In [18]:
def itemcf_sim(click_df, item_created_time_dict, option='raw'):
    """
    Item similarity matrix calculation.
    :param df: DataFrame
    :item_created_time_dict: Dictionary of article creation times
    :return: The article-to-article similarity matrix

    Approach: Item-based Collaborative Filtering + Association Rules
    """
    # Get a dictionary of items which clicked by users
    # Key is the user ID, value is the list of items that the user clicked.
    user_item_time_dict = get_user_item_time(click_df)

    # Initialize dictionaries
    i2i_sim = defaultdict(dict)
    item_cnt = defaultdict(int)

    if option == 'raw':
        print('Raw similarity calculation\n')
        # Raw similarity accumulation process
        # Iterate over each user and their list of clicked items and times
        for user, item_time_list in tqdm(user_item_time_dict.items(),
                                         desc="Building item-item similarity matrix (raw)"):
            # Iterate over each item i clicked by the user
            for i, i_click_time in item_time_list:
                item_cnt[i] += 1  # Record the number of times item i was clicked

                # Nested iteration over each pair of items i and j clicked by the user
                for j, j_click_time in item_time_list:
                    if i == j:  # Skip if item i and item j are the same
                        continue

                    # Initialize similarity between i and j to 0 if not already set
                    i2i_sim[i].setdefault(j, 0)                   
                    # Accumulate similarity between item i and item j
                    i2i_sim[i][j] += 1 / math.log(len(item_time_list) + 1)
                    
    elif option == 'weighted':
        print('Weighted similarity calculation\n')
        for user, item_time_list in tqdm(user_item_time_dict.items(),
                                         desc="Building item-item similarity matrix (weighted)"):
            # Iterate over each item i clicked by the user
            for loc1, (i, i_click_time) in enumerate(item_time_list):
                item_cnt[i] += 1  # Record the number of times item i was clicked
                i2i_sim.setdefault(i, {})
                
                # Nested iteration over each pair of items i and j clicked by the user
                for loc2, (j, j_click_time) in enumerate(item_time_list):
                    if i == j:  # Skip if item i and item j are the same
                        continue

                    # Consider both forward and backward sequential clicks
                    loc_alpha = 1.0 if loc2 > loc1 else 0.7
                    # Positional weight; parameters can be adjusted
                    loc_weight = loc_alpha * (0.9 ** (np.abs(loc2 - loc1) - 1))
                    # Click time weight; parameters can be adjusted
                    click_time_weight = np.exp(0.7 ** np.abs(i_click_time - j_click_time))
                    # Article creation time weight; parameters can be adjusted
                    created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
                    # Initialize similarity between i and j to 0 if not already set
                    i2i_sim[i].setdefault(j, 0)                   
                    # Accumulate similarity between item i and item j considering multiple factors
                    i2i_sim[i][j] += loc_weight * click_time_weight * created_time_weight / math.log(len(item_time_list) + 1)
    
    # Normalize the similarity values  
    i2i_sim_final = defaultdict(dict)
    for i, related_items in i2i_sim.items():
        for j, wij in related_items.items():
            i2i_sim_final[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])

    # Save the similarity matrix to local path
    with open(save_path + 'itemcf_i2i_sim.pkl', 'wb') as f:
        pickle.dump(i2i_sim_final, f)

    return i2i_sim_final

#### About Similarity Calculation

1. **Similarity Accumulation Process**:

   $$
   \text{sim}(i, j) = \sum_{u \in U_{ij}} \frac{1}{\log(1 + |N(u)|)}
   $$

   Where:
   - $ \text{sim}(i, j) $ represents the similarity between item $ i $ and item $ j $.
   - $ U_{ij} $ denotes the set of users who clicked both item $ i $ and item $ j $.
   - $ N(u) $ refers to the number of items clicked by user $ u $.

2. **Similarity Normalization Process**:

   $$
   \text{sim\_normalized}(i, j) = \frac{\text{sim}(i, j)}{\sqrt{c(i) \times c(j)}}
   $$

   Where:
   - $ \text{sim\_normalized}(i, j) $ denotes the normalized similarity between item $ i $ and item $ j $.
   - $ c(i) $ and $ c(j) $ represent the total number of clicks for item $ i $ and item $ j $, respectively.

Similarity is calculated based on co-occurrence frequency. `1 / log(len(item_time_list) + 1)` or `weight / log(len(item_time_list) + 1)` is used as the increment value for similarity to standardize across different lengths of user click lists.

- **Co-occurrence Frequency**: The similarity value reflects how often two items appear together in the same user’s click history. The more frequent the co-occurrence, the higher the similarity.
- **Normalization**: `1 / math.log(len(item_time_list) + 1)` normalizes the similarity increment to reduce the influence of long click lists on the calculation.

The weighted similarity considers:
- **Positional Weight** `loc_weight = loc_alpha * (0.9 ** (np.abs(loc2 - loc1) - 1))`: This weight accounts for the order and distance of a user's clicks. `loc_alpha = 1.0 if loc2 > loc1 else 0.7` is a directional modifier. It assigns a higher value (1.0) if a user clicks article j after article i, and a lower value (0.7) if the order is reversed. This gives more importance to the most recent behavior. The positional weight decays exponentially based on the distance between the clicks `np.abs(loc2 - loc1)`. The further apart the two clicks are in a user's history, the smaller this weight becomes. The 0.9 is a decay coefficient that controls how quickly this weight diminishes.
- **Click Time Weight** `click_time_weight = np.exp(0.7 ** np.abs(i_click_time - j_click_time))`: This weight is based on how much time passed between a user's clicks on articles i and j. The closer the clicks are in time, the higher the weight. This assumes that a user's interests are more similar for items they engage with in quick succession. The 0.7 is a decay coefficient that controls the speed of this decay.
- **Article Creation Time Weight** `created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))`: This weight considers the freshness of the articles themselves. It gives a higher weight to articles that were created around the same time. This can capture trends where users are interested in a specific topic that is currently being published. The 0.8 is the decay coefficient.
- **Final Similarity Score** `i2i_sim[i][j] += loc_weight * click_time_weight * created_time_weight / math.log(len(item_time_list) + 1)`: The final similarity score `i2i_sim[i][j]` is a composite of the above factors. It's calculated by multiplying the three weights together (loc_weight * click_time_weight * created_time_weight) and then dividing by a logarithmic term math.log(len(item_time_list) + 1). This final division is a normalization step to prevent a single user's frequent clicks from disproportionately inflating the similarity score.

In [19]:
i2i_sim = itemcf_sim(all_click_df, item_created_time_dict, option='weighted')

Weighted similarity calculation



Building item-item similarity matrix (weighted): 100%|███████████████████████| 200000/200000 [02:27<00:00, 1358.32it/s]


### 6.2 User Collaborative Filtering - user2user similarity 
We then obtain the `user2user` similarity matrix based on user-based collaborative filtering (userCF). When calculating the user-to-user similarity matrix, some simple association rules can also be applied, such as **user activity weight**. Here, the number of user clicks is used as an indicator of user activity.

In [20]:
def get_user_active_degree_dict(click_df):
    click_df_ = click_df.groupby('user_id')['click_article_id'].count().reset_index()

    # Normalize user activity
    mm = MinMaxScaler()
    click_df_['click_article_id'] = mm.fit_transform(click_df_[['click_article_id']])
    user_active_degree_dict = dict(zip(click_df_['user_id'], click_df_['click_article_id']))

    return user_active_degree_dict

In [21]:
def usercf_sim(click_df, user_active_degree_dict):
    """
    User similarity matrix calculation.
    :param all_click_df: DataFrame
    :param user_activate_degree_dict: Dictionary of user activity levels
    :return: User similarity matrix
    """
    # Get a dictionary of users who clicked on each item.
    # Key is the item ID, value is the list of users who clicked the item.
    item_user_time_dict = get_item_user_time_dict(click_df)

    # Initialize dictionaries
    u2u_sim = defaultdict(dict)
    user_cnt = defaultdict(int)

    # Iterate over each item and its list of users who clicked it
    for item, user_time_list in tqdm(item_user_time_dict.items()):
        # Iterate over each user and their click time
        for u, u_click_time in user_time_list:
            # Count the number of items clicked by each user
            user_cnt[u] += 1
            # Nested iteration over users to calculate user similarity
            for v, v_click_time in user_time_list:
                # Skip if the two users are the same (no self-similarity calculation)
                if u == v:
                    continue
                # Initialize the similarity value for users if not already set
                u2u_sim[u].setdefault(v, 0)
                # Calculate the similarity weight considering user activity
                active_weight = 100 * 0.5 * (user_active_degree_dict[u] + user_active_degree_dict[v])
                # Update the similarity value in the matrix for the two users
                u2u_sim[u][v] += active_weight / math.log(len(user_time_list) + 1)

    # Copy the user similarity matrix to avoid modifying the original data 
    u2u_sim_ = u2u_sim.copy()
    # Normalize the similarity values in the user similarity matrix
    for u, related_users in u2u_sim.items():
        for v, wij in related_users.items():
            u2u_sim_[u][v] = wij / math.sqrt(user_cnt[u] * user_cnt[v])
                
    # Save the user similarity matrix to the local storage
    pickle.dump(u2u_sim_, open(save_path + 'usercf_u2u_sim.pkl', 'wb'))

    return u2u_sim_       

In [23]:
# Calculating UserCF is too memory-intensive, so we use a sampled dataset to run.
user_active_degree_dict = get_user_active_degree_dict(all_click_df)
u2u_sim = usercf_sim(all_click_df[:20000], user_active_degree_dict)

100%|█████████████████████████████████████████████████████████████████████████████| 1735/1735 [00:10<00:00, 159.11it/s]


### 6.3 Item Embedding Similarity
Item embedding similarity is a method for finding articles that are semantically or contextually similar to each other. Every article is represented by a unique vector (an "embedding") in a high-dimensional space. The closer two vectors are in this space, the more similar the articles they represent.

This is particularly useful for new articles that have no click data (a cold-start scenario). By comparing a new article's embedding to the embeddings of all other articles in the database, we can find a set of similar articles that can be recommended to users who have previously shown an interest in that topic.

#### An Introduction to Faiss
Faiss is a core component in modern recommendation systems. Developed by Facebook's AI team, it is a high-performance library built for efficient similarity search and clustering of dense vectors.

In a large-scale recommendation system, simply calculating the similarity between every item and every other item taking a prohibitive amount of time during online inference. Faiss solves this problem by using advanced algorithms that can perform an approximate nearest neighbor search (ANN).

This means that instead of finding the perfect nearest neighbor, Faiss finds a vector that is a very close approximation, and it does so in a fraction of the time. This trade-off between absolute accuracy and speed is essential for real-world applications where a massive search must be completed quickly.

**The key benefit of Faiss is its ability to find the top-k most similar vectors to a given query vector at an incredible speed**, making it the perfect tool for the vector recall stage of a recommendation system.

#### **Principle of Faiss Query:**

1. **Index Structure Construction:**  
   In Faiss, the first step is to build an index structure for the vector dataset to be searched. Faiss provides various index structures, such as flat indexes, inverted file indexes (IVF), and product quantization indexes (PQ). Each index structure is suited to specific scenarios and requirements, allowing users to choose the most appropriate one based on their needs.

2. **Vector Encoding:**  
   Before constructing the index structure, vector data typically needs to be encoded. The purpose of encoding is to map high-dimensional vectors into a lower-dimensional space to reduce computation and storage costs. Common encoding methods include vector quantization and hash encoding.

3. **Query Process:**  
   Once the index structure is built, queries can be performed on it. The query process involves calculating the similarity between the query vector and the vectors stored in the index structure. Faiss provides multiple query methods, such as exact search and approximate search. In approximate search, Faiss uses algorithms such as Product Quantization (PQ) and Locality Sensitive Hashing (LSH) to accelerate the search process.

4. **Result Retrieval:**  
   After the query is complete, Faiss returns the indexes or distances of the top-k most similar vectors. Users can specify the number of results they need and process or analyze the returned results further.

---

1. **Product Quantization (PQ):**  
   - PQ divides a high-dimensional vector into multiple sub-vectors and independently quantizes each sub-vector. This maps the original high-dimensional vector into a lower-dimensional subspace, reducing computation and storage requirements.
   - The key to PQ lies in how the original vector is divided into sub-vectors and how appropriate quantization methods are designed. Common methods include Product Quantization and Residual Product Quantization.
   - PQ significantly reduces search time while maintaining search quality, making it especially useful for large-scale high-dimensional data in approximate nearest neighbor searches.

2. **Locality Sensitive Hashing (LSH):**  
   - LSH is a hash-based method for approximate nearest neighbor search. Its core idea is to map similar vectors into the same bucket so that similar vectors can be quickly located during the query.
   - LSH uses multiple hash functions to generate several hash values for each vector, which are then used to map the vectors into buckets. During a query, only the vectors in the query vector’s bucket and neighboring buckets are searched, eliminating the need to traverse the entire dataset.
   - LSH provides high search efficiency and scalability when dealing with large-scale data, particularly for high-dimensional data in approximate nearest neighbor searches.

---

Faiss utilizes two key techniques for vector compression and encoding: **PCA** (Principal Component Analysis) and **PQ** (Product Quantization), along with other optimization techniques. However, PCA and PQ are the core components.

In [24]:
# Vector Retrieval Similarity Calculation
# topk refers to the number of most similar items returned by Faiss after searching for each individual item.
def embedding_sim(item_emb_df, save_path, topk):
    """
    Content-based article embedding similarity matrix calculation.
    :param click_df: DataFrame containing click data.
    :param item_emb_df: DataFrame containing article embeddings.
    :param save_path: Path to save the similarity matrix.
    :param topk: Number of most similar articles to retrieve.
    :return: Article similarity matrix.

    Approach: For each article, return the top-k most similar articles based on embedding similarity.
    Since the number of articles is large, Faiss is used to accelerate the process.
    """
    # Dictionary mapping between article index and article ID
    item_idx_2_rawid_dict = dict(zip(item_emb_df.index, item_emb_df['article_id']))

    # Extract embedding columns
    item_emb_cols = [x for x in item_emb_df.columns if 'emb' in x]
    item_emb_np = np.ascontiguousarray(item_emb_df[item_emb_cols].values, dtype=np.float32)

    # Normalize the vectors
    item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)

    # Build Faiss index
    item_index = faiss.IndexFlatIP(item_emb_np.shape[1])  # Inner Product index, embedding vector dimension as parameter
    item_index.add(item_emb_np)  # Add embeddings to the index

    # Perform similarity search for each article, returning top-k similar vectors
    sim, idx = item_index.search(item_emb_np, topk)

    # Dictionary to store similarity results with original article IDs
    item_sim_dict = defaultdict(dict)

    # Add progress bar using tqdm
    for target_idx, sim_value_list, rele_idx_list in tqdm(zip(range(len(item_emb_np)), sim, idx)):
        target_raw_id = item_idx_2_rawid_dict[target_idx]
        # Start from 1 to exclude the article itself, so the final result contains topk-1 similar articles
        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]):
            rele_raw_id = item_idx_2_rawid_dict[rele_idx]
            item_sim_dict[target_raw_id][rele_raw_id] = item_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value

    # Save the item-to-item similarity matrix as a pickle file
    pickle.dump(item_sim_dict, open(save_path + 'emb_i2i_sim.pkl', 'wb'))

    return item_sim_dict

In [31]:
item_emb_df = pd.read_csv(data_path + 'articles_emb.csv')
emb_i2i_sim = embedding_sim(item_emb_df, save_path, topk=10) 

364047it [00:12, 28462.65it/s]


## 7. The Recall Phase

The recall phase is a crucial first step in a recommendation system. Its job is to quickly narrow down a vast number of items to a smaller, more manageable set of potential candidates that a user might be interested in.

#### How Recall Works
Imagine you're on a shopping website. The system quickly looks at your browsing history, past purchases, and other data to find a large pool of products you might like. This could mean pulling up all electronics because you recently bought a new laptop, or all items in a specific category you frequently browse.

The goal isn't to find the perfect item yet, but to efficiently find a wide array of possibilities. These candidates are then passed on to the next step, the ranking phase, which will score and sort them to find the best few to show you.

This two-step approach saves a lot of computing power and makes the entire recommendation process much faster and more efficient.

#### Strategies for Scaling Down the Problem
The recall phase addresses the problem we discussed at the beginning: how do we reduce the scale of a problem involving 360,000 articles and over 200,000 users? We can use the recall phase to filter a set of candidate articles for each user, thus reducing the problem's scale. Common recall strategies include:

- **YouTube DNN Recall**: A deep learning approach that uses a user's past behavior (like watch and click history) to predict which articles they are most likely to click on. This is a very effective way to narrow down a huge list of candidates.

- **Item-based Collaborative Filtering**: This method finds articles that are similar to the ones a user has already clicked on, and adds these similar articles to the candidate set.

- **User-based Collaborative Filtering**: This method finds other users who have similar interests and behavior as the user. It then recommends articles that those similar users have engaged with.

- **Content-based Filtering**: This approach uses article embedding vectors to measure the similarity between articles, and then performs recall based on this similarity. For example, models like Word2Vec or Doc2Vec can provide these embeddings.

- **User Embedding**: Similar to content-based filtering, this method represents each user as a vector. It then finds other users with similar vectors and recommends their content. This is a powerful way to make personalized recommendations.

Each of these methods provides a unique way to calculate similarity, whether it's based on user behavior, article content, or a combination of both. By combining these strategies, we can create a robust and efficient system that provides a strong foundation for the final recommendation process.

### 7.1 YoutubeDNN Recall
Covington et al. 2016 (https://static.googleusercontent.com/media/research.google.com/zh-CN//pubs/archive/45530.pdf)
![](https://wuciawe.github.io/files/2019-05-15-notes-on-youtube-dnn/candidate_generation.png)

YoutubeDNN uses a negative sampling technique to train the model. The below function is for generating training and test data. A small set of negative samples (articles not clicked by the user) will be chosen for each positive sample (articles clicked by the user). 

This function is designed to generate training and test sets from a dataset of user clicks. It takes the following parameters:
- data: A DataFrame containing the raw click data.
- negsample: The number of negative samples to be chosen from unclicked articles when building the training windows.

The generating function works as follows:
1. Sorts the data by click time to ensure correct chronological order.
2. Gathers a list of all unique article IDs.
3. Initializes empty lists to store the training and test sets.
4. Groups the data by user_id to process each user individually.
5. For each user, it gets their list of clicked articles to serve as positive samples.
6. If negsample is greater than zero, it randomly selects negative samples from the pool of articles the user has not clicked.
7. If a user has only clicked one article, that click is added to both the training and test sets.
8. It uses a sliding window method to create positive and negative sample pairs. The last click in the sequence for each user is reserved for the test set.
9. Randomly shuffles the order of the training and test sets.
10. Finally, it returns the generated training and test data.

In [32]:
# Generate Training and Validation Data for Two-Tower Recall
# negsample refers to the number of negative samples chosen when constructing the training samples using a sliding window
def gen_data_set(data, negsample=0):
    data.sort_values("click_timestamp", inplace=True)
    item_ids = data['click_article_id'].unique()

    train_set = []
    test_set = []
    for userID, hist in tqdm(data.groupby('user_id')):
        pos_list = hist['click_article_id'].tolist()  # positive sample list

        if negsample > 0:
            candidate_set = list(set(item_ids) - set(pos_list))  # Select negative samples from articles the user has not seen.
            neg_list = np.random.choice(candidate_set,size=len(pos_list)*negsample,replace=True) # For each positive sample, select n negative samples.
            
        # When the sequence length is only one, this data point must also be included in the training set. Otherwise, the final learned embeddings will have missing values.
        if len(pos_list) == 1:
            train_set.append((userID, [pos_list[0]], pos_list[0], 1, len(pos_list)))
            test_set.append((userID, [pos_list[0]], pos_list[0], 1, len(pos_list)))

        # Constructing positive and negative samples using a sliding window.
        for i in range(1, len(pos_list)):
            hist = pos_list[:i]

            if i != len(pos_list) - 1:
                train_set.append((userID, hist[::-1], pos_list[i], 1, len(hist[::-1])))  # Positive Sample [user_id, his_item, pos_item, label, len(his_item)]
                for negi in range(negsample):
                    train_set.append((userID, hist[::-1], neg_list[i*negsample+negi], 0, len(hist[::-1])))  # negative sample [user_id, his_item, neg_item, label, len(his_item)]
                else:
                    # Use the longest sequence length as the test data.
                    test_set.append((userID, hist[::-1], pos_list[i], 1, len(hist[::-1])))
    
    random.shuffle(train_set)
    random.shuffle(test_set)
    
    return train_set, test_set

The below function is for Padding and Preparing Model Input. This function is used to pad the input data so that all sequence features have a consistent length, which is a requirement for many deep learning models. It takes these parameters:

- train_set: The training samples generated by the gen_data_set function.
- user_profile: A DataFrame containing user attribute information.
- seq_max_len: The maximum length for the sequence features.

The function's main steps are:
1. Extracts user IDs, historical article sequences, clicked article IDs, labels, and historical sequence lengths from the train_set.
2. Pads the historical article sequences to ensure they all have the same length, defined by seq_max_len.
3. Builds a dictionary of model inputs, including the user ID, clicked article ID, padded historical sequence, and the original historical sequence length.
4. Returns the model input dictionary and its corresponding labels.

In [33]:
# Pad the input data to ensure that all sequence features have a uniform length
def gen_model_input(train_set, seq_max_len):
    train_uid = np.array([line[0] for line in train_set])
    train_seq = [line[1] for line in train_set]
    train_iid = np.array([line[2] for line in train_set])
    train_label = np.array([line[3] for line in train_set])
    train_hist_len = np.array([line[4] for line in train_set])

    train_seq_pad = pad_sequences(train_seq, maxlen=seq_max_len, padding='post', truncating='post', value=0)
    train_model_input = {"user_id": train_uid, "click_article_id": train_iid, "hist_article_id": train_seq_pad,
                         "hist_len": train_hist_len}

    return train_model_input, train_label

In [40]:
def youtubednn_u2i_dict(data, topk=20):
    SEQ_LEN = 30 # For the user click sequence length, pad the short sequences and truncate the long ones.

    user_profile_ = data[["user_id"]].drop_duplicates('user_id')
    item_profile_ = data[["click_article_id"]].drop_duplicates('click_article_id')

    # Label Encoding
    features = ["click_article_id", "user_id"]
    feature_max_idx = {}

    for feature in features:
        lbe = LabelEncoder()
        data[feature] = lbe.fit_transform(data[feature])
        feature_max_idx[feature] = data[feature].max() + 1

    # Extracting user and item profiles. The specific features to select here require further analysis and consideration.
    user_profile = data[["user_id"]].drop_duplicates('user_id')
    item_profile = data[["click_article_id"]].drop_duplicates('click_article_id')

    user_index_2_rawid = dict(zip(user_profile['user_id'], user_profile_['user_id']))
    item_index_2_rawid = dict(zip(item_profile['click_article_id'], item_profile_['click_article_id']))

    # Splitting into Training and Test Sets
    # Because deep learning typically requires a very large amount of data, we often expand the training samples using a sliding window approach to ensure a high-quality recall performance.
    train_set, test_set = gen_data_set(data, 1)

    # Generate model input
    train_model_input, train_label = gen_model_input(train_set, SEQ_LEN)
    test_model_input, test_label = gen_model_input(test_set, SEQ_LEN)

    # Dimension of embedding
    embedding_dim = 16

    # Organize the data into a format that the model can directly accept.
    user_feature_columns = [SparseFeat('user_id', feature_max_idx['user_id'], embedding_dim),
                            VarLenSparseFeat(SparseFeat('hist_article_id', feature_max_idx['click_article_id'], embedding_dim,
                                                        embedding_name="click_article_id"), SEQ_LEN, 'mean', 'hist_len'),]
    item_feature_columns = [SparseFeat('click_article_id', feature_max_idx['click_article_id'], embedding_dim)]

    train_counter = Counter(train_model_input['click_article_id'])
    item_count = [train_counter.get(i, 0) for i in range(item_feature_columns[0].vocabulary_size)]
    sampler_config = NegativeSampler('frequency', num_sampled=5, item_name="click_article_id", item_count=item_count)

    if tf.__version__ >= '2.0.0':
        tf.compat.v1.disable_eager_execution()
    else:
        K.set_learning_phase(True)
        
    # Model Definition
    # num_sampled: The number of samples to use during negative sampling.
    model = YoutubeDNN(user_feature_columns, item_feature_columns, user_dnn_hidden_units=(64, embedding_dim), sampler_config=sampler_config)
    
    # Compile Model
    model.compile(optimizer="adam", loss=sampledsoftmaxloss)

    # Model Training
    # The proportion of the validation can be defined by validation_split. If it is set to 0, the model will be trained directly on the full dataset.
    history = model.fit(train_model_input, train_label, batch_size=1024, epochs=30, verbose=1, validation_split=0.2)

    # After the model has finished training, extract the learned embeddings, including both the user and item embeddings.
    test_user_model_input = test_model_input
    all_item_model_input = {"click_article_id": item_profile['click_article_id'].values}

    user_embedding_model = Model(inputs=model.user_input, outputs=model.user_embedding)
    item_embedding_model = Model(inputs=model.item_input, outputs=model.item_embedding)

    # Save the current item and user embeddings. They might be useful for the ranking phase, but be sure to save them in a way that corresponds to their original IDs.
    user_embs = user_embedding_model.predict(test_user_model_input, batch_size=2 ** 12)
    item_embs = item_embedding_model.predict(all_item_model_input, batch_size=2 ** 12)

    # Normalize the embeddings before saving.
    user_embs = user_embs / np.linalg.norm(user_embs, axis=1, keepdims=True)
    item_embs = item_embs / np.linalg.norm(item_embs, axis=1, keepdims=True)

    # Convert the embeddings into a dictionary format for easy querying.
    raw_user_id_emb_dict = {user_index_2_rawid[k]: \
                                v for k, v in zip(user_profile['user_id'], user_embs)}
    raw_item_id_emb_dict = {item_index_2_rawid[k]: \
                                v for k, v in zip(item_profile['click_article_id'], item_embs)}

    # Save the embeddings locally.
    pickle.dump(raw_user_id_emb_dict, open(save_path + 'user_youtube_emb.pkl', 'wb'))
    pickle.dump(raw_item_id_emb_dict, open(save_path + 'item_youtube_emb.pkl', 'wb'))

    # Faiss nearest neighbor search: use user embeddings to search for the top-k most similar items
    index = faiss.IndexFlatIP(embedding_dim)
    # Normalization has already been performed above, so we can skip it here
    #faiss.normalize_L2(user_embs)
    #faiss.normalize_L2(item_embs)
    index.add(item_embs)  # Build an index from the item vectors
    sim, idx = index.search(np.ascontiguousarray(user_embs), topk)  # Query with user vectors to find the top-k most

    user_recall_items_dict = defaultdict(dict)
    for target_idx, sim_value_list, rele_idx_list in tqdm(zip(test_user_model_input['user_id'], sim, idx)):
        target_raw_id = user_index_2_rawid[target_idx]
        # Starting from 1 is to remove the item itself, so the final similar items obtained are topk-1
        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]):
            rele_raw_id = item_index_2_rawid[rele_idx]
            user_recall_items_dict[target_raw_id][rele_raw_id] = user_recall_items_dict.get(target_raw_id, {})\
                                                                    .get(rele_raw_id, 0) + sim_value
    user_recall_items_dict = {k: sorted(v.items(), key=lambda x: x[1], reverse=True) for k, v in user_recall_items_dict.items()}
    # Save the recall results
    # Here, we get the recall results directly via the vector approach.
    # This is different from the methods above, which only produced i2i and u2u similarity matrices
    # and still required a collaborative filtering recall step to get the results.
    # These recall results can be directly evaluated. For convenience, a single
    # evaluation function can be written to evaluate all recall results.
    pickle.dump(user_recall_items_dict, open(save_path + 'youtube_u2i_recall_dict.pkl', 'wb'))
    return user_recall_items_dict

#### YoutubeDNN recall and evaluation

In [41]:
# Since we need to evaluate recall here, we have extracted the last click from the training set.
metric_recall = True

if not metric_recall:
    user_multi_recall_dict['youtubednn_recall'] = youtubednn_u2i_dict(all_click_df, topk=50)
else:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
    user_multi_recall_dict['youtubednn_recall'] = youtubednn_u2i_dict(trn_hist_click_df, topk=50)
    # Evaluate recall performance
    metrics_recall(user_multi_recall_dict['youtubednn_recall'], trn_last_click_df, topk=50)

100%|█████████████████████████████████████████████████████████████████████████| 200000/200000 [32:33<00:00, 102.36it/s]


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 1016128 samples, validate on 254032 samples
Epoch 1/30
1016128/1016128 [==============================] - 169s 166us/sample - loss: 3.9269 - val_loss: 3.3179
Epoch 2/30
1016128/1016128 [==============================] - 183s 180us/sample - loss: 3.3996 - val_loss: 3.5891
Epoch 3/30
1016128/1016128 [==============================] - 184s 181us/sample - loss: 3.2787 - val_loss: 3.8667
Epoch 4/30
1016128/1016128 [==============================] - 179s 176us/sample - loss: 3.0007 - val_loss: 3.8141
Epoch 5/30
1016128/1016128 [==============================] - 173s 171us/sample - loss: 2.7218 - val_loss: 3.9326
Epoch 6/30
1016128/1016128 [==============================] - 161s 158us/sample - loss: 2.5224 - val_loss: 3.8504
Epoch 7/30
1016128/1016128 [==============================] - 166s 163us/sample - loss: 2.2885 - val_loss: 3.6770
Epoch 8/30
1016128/1016128 [===

675899it [01:14, 9098.14it/s] 


topk: 10  | hit_num: 75  | hit_rate: 0.00041  | user_num: 181592
topk: 20  | hit_num: 142  | hit_rate: 0.00078  | user_num: 181592
topk: 30  | hit_num: 199  | hit_rate: 0.0011  | user_num: 181592
topk: 40  | hit_num: 257  | hit_rate: 0.00142  | user_num: 181592
topk: 50  | hit_num: 308  | hit_rate: 0.0017  | user_num: 181592


### 7.2 Item-based Collaborative Filtering Recall
Above, we used collaborative filtering and embedding retrieval to obtain similarity matrices for articles. Below, we'll apply the concept of collaborative filtering to recall articles that are similar to a user's historical clicks.

In this recall process, we also use an association rule-based approach:

1. Weight based on the order of similar and previously clicked articles: The order in which a user clicks on articles is considered for weighting.

2. Weight based on article creation time: We assign a weight based on the time difference between the creation dates of the similar article and the historically clicked article.

3. Weight based on article content similarity: Embeddings are used to calculate the similarity between articles. However, it is important to note that embeddings are not calculated for every possible pair of items. Therefore, special handling is required if a similar article and a historically clicked article do not have a pre-computed similarity score.

In [42]:
# Item-based Collaborative Filtering Recall (i2i)
def item_based_recommend(user_id, user_item_time_dict, i2i_sim, sim_item_topk, recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim):
    """
    Performs item-based collaborative filtering for recall.
    :param user_id: The user's ID.
    :param user_item_time_dict: A dictionary of user's clicked article sequences, keyed by user ID.
                                 Example: {user1: [(item1, time1), (item2, time2)..]...}
    :param i2i_sim: A dictionary representing the article similarity matrix.
    :param sim_item_topk: An integer, selecting the top-k most similar articles to the current article.
    :param recall_item_num: An integer, the final number of articles to recall.
    :param item_topk_click: A list of the most-clicked articles, used to fill in if not enough candidates are found.
    :param emb_i2i_sim: A dictionary for the article similarity matrix based on content embeddings.

    :return: A list of recalled articles and their scores: [(item1, score1), (item2, score2)...]
    """
    # Get the user's historical interaction articles
    user_hist_items = user_item_time_dict[user_id]
    user_hist_items_ = {user_id for user_id, _ in user_hist_items}

    item_rank = {}
    for loc, (i, click_time) in enumerate(user_hist_items):
        for j, wij in sorted(i2i_sim[i].items(), key=lambda x: x[1], reverse=True)[:sim_item_topk]:
            if j in user_hist_items_:
                continue
            # Weight based on the difference in article creation times
            created_time_weight = np.exp(0.8 ** np.abs(item_created_time_dict[i] - item_created_time_dict[j]))
            # Weight based on the position of the historical article in the user's sequence
            loc_weight = (0.9 ** (len(user_hist_items) - loc))

            content_weight = 1.0
            if emb_i2i_sim.get(i, {}).get(j, None) is not None:
                content_weight += emb_i2i_sim[i][j]
            if emb_i2i_sim.get(j, {}).get(i, None) is not None:
                content_weight += emb_i2i_sim[j][i]

            item_rank.setdefault(j, 0)
            item_rank[j] += created_time_weight * loc_weight * content_weight * wij

    # If there are fewer than the required number of items, fill with popular items
    if len(item_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in item_rank.items():  # The filled item should not be in the original list
                continue
            item_rank[item] = - i - 100   # Assign a low negative score
            if len(item_rank) == recall_item_num:
                break

    item_rank = sorted(item_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]

    return item_rank

#### itemcf sim recall and evaluation

In [44]:
# First, perform Item-based Collaborative Filtering (itemcf) recall. 
# The last click is extracted for the purpose of recall evaluation.
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)

i2i_sim = pickle.load(open(save_path + 'itemcf_i2i_sim.pkl', 'rb'))
emb_i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl', 'rb'))

sim_item_topk = 50
recall_item_num = 50
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, \
                                                        i2i_sim, sim_item_topk, recall_item_num, \
                                                        item_topk_click, item_created_time_dict, emb_i2i_sim)

user_multi_recall_dict['itemcf_sim_itemcf_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['itemcf_sim_itemcf_recall'], open(save_path + 'itemcf_recall_dict.pkl', 'wb'))

if metric_recall:
    # Evaluate recall performance
    metrics_recall(user_multi_recall_dict['itemcf_sim_itemcf_recall'], trn_last_click_df, topk=recall_item_num)

100%|█████████████████████████████████████████████████████████████████████████| 200000/200000 [30:42<00:00, 108.58it/s]


topk: 10  | hit_num: 97853  | hit_rate: 0.48927  | user_num: 200000
topk: 20  | hit_num: 123349  | hit_rate: 0.61674  | user_num: 200000
topk: 30  | hit_num: 136999  | hit_rate: 0.685  | user_num: 200000
topk: 40  | hit_num: 146051  | hit_rate: 0.73025  | user_num: 200000
topk: 50  | hit_num: 152664  | hit_rate: 0.76332  | user_num: 200000


### 7.3 Item Embedding Similarity Recall

In [45]:
# The last click is extracted for the purpose of recall evaluation.
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)
i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl','rb'))

sim_item_topk = 50
recall_item_num = 50
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, sim_item_topk,
                                                        recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim)

user_multi_recall_dict['embedding_sim_item_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['embedding_sim_item_recall'], open(save_path + 'embedding_sim_recall_dict.pkl', 'wb'))

if metric_recall:
    # Evaluate recall performance
    metrics_recall(user_multi_recall_dict['embedding_sim_item_recall'], trn_last_click_df, topk=recall_item_num)

100%|████████████████████████████████████████████████████████████████████████| 200000/200000 [00:59<00:00, 3347.94it/s]


topk: 10  | hit_num: 3986  | hit_rate: 0.01993  | user_num: 200000
topk: 20  | hit_num: 12376  | hit_rate: 0.06188  | user_num: 200000
topk: 30  | hit_num: 19134  | hit_rate: 0.09567  | user_num: 200000
topk: 40  | hit_num: 26550  | hit_rate: 0.13275  | user_num: 200000
topk: 50  | hit_num: 33422  | hit_rate: 0.16711  | user_num: 200000


### 7.4 User-based Collaborative Filtering Recall
The core idea behind user-based collaborative filtering is to recommend articles to a user that were clicked by other similar users. Since this involves the historical articles of similar users, we can still add some association rules to weight the articles that are likely to be clicked. The association rules used here primarily consider the relationship between the historical articles of similar users and the articles of the user being recommended to. The relationship here can directly borrow from the approach used in item-based collaborative filtering. This process accumulates weights for the items being recommended.

Here are the relationships used for weighting, along with the corresponding code:

- Calculate the similarity, creation time difference, and relative position sum between the articles clicked by the user being recommended to and the articles clicked by similar users. This sum serves as their respective weights.

Note: Calculating user-to-user similarity in UserCF is too memory-intensive. Therefore, we did not run it on the full dataset and instead used a sampled dataset.

In [46]:
# User-based Collaborative Filtering Recall (u2u)
def user_based_recommend(user_id, user_item_time_dict, u2u_sim, sim_user_topk, recall_item_num,
                         item_topk_click, item_created_time_dict, emb_i2i_sim):
    """
    Performs user-based collaborative filtering for recall.
    :param user_id: The user's ID.
    :param user_item_time_dict: A dictionary, keyed by user ID, containing the user's clicked article sequence based on click time.
                                 Example: {user1: [(item1, time1), (item2, time2)..]...}
    :param u2u_sim: A dictionary representing the user similarity matrix.
    :param sim_user_topk: An integer, selecting the top-k most similar users to the current user.
    :param recall_item_num: An integer, the final number of articles to recall.
    :param item_topk_click: A list of the most-clicked articles, used to fill in if not enough candidates are found.
    :param item_created_time_dict: A dictionary of article creation times.
    :param emb_i2i_sim: A dictionary for the article similarity matrix based on content embeddings.

    :return: A list of recalled articles and their scores: [(item1, score1), (item2, score2)...]
    """
    # Get user's historical interactions
    user_item_time_list = user_item_time_dict[user_id]    #  [(item1, time1), (item2, time2)..]
    user_hist_items = set([i for i, t in user_item_time_list])  # A user may interact with an article multiple times; this removes duplicates.

    items_rank = {}
    for sim_u, wuv in sorted(u2u_sim[user_id].items(), key=lambda x: x[1], reverse=True)[:sim_user_topk]:
        for i, click_time in user_item_time_dict[sim_u]:
            if i in user_hist_items:
                continue
            items_rank.setdefault(i, 0)

            loc_weight = 1.0
            content_weight = 1.0
            created_time_weight = 1.0

            # Apply association rules based on the user's historical articles
            for loc, (j, click_time) in enumerate(user_item_time_list):
                # Weight based on the relative position of the historical article
                loc_weight += 0.9 ** (len(user_item_time_list) - loc)
                # Weight based on content similarity
                if emb_i2i_sim.get(i, {}).get(j, None) is not None:
                    content_weight += emb_i2i_sim[i][j]
                if emb_i2i_sim.get(j, {}).get(i, None) is not None:
                    content_weight += emb_i2i_sim[j][i]

                # Weight based on creation time difference
                created_time_weight += np.exp(0.8 * np.abs(item_created_time_dict[i] - item_created_time_dict[j]))

            items_rank[i] += loc_weight * content_weight * created_time_weight * wuv

    # If there are not enough recalled items, fill with popular items
    if len(items_rank) < recall_item_num:
        for i, item in enumerate(item_topk_click):
            if item in items_rank.items():  # The filled item should not be in the original list
                continue
            items_rank[item] = - i - 100  # Assign a low negative score
            if len(items_rank) == recall_item_num:
                break

    items_rank = sorted(items_rank.items(), key=lambda x: x[1], reverse=True)[:recall_item_num]

    return items_rank

#### usercf sim recall and evaluation

In [ ]:
# The last click is extracted for the purpose of recall evaluation.
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)

u2u_sim = pickle.load(open(save_path + 'usercf_u2u_sim.pkl', 'rb'))

sim_user_topk = 20
recall_item_num = 10
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = user_based_recommend(user, user_item_time_dict, u2u_sim, sim_user_topk, \
                                                        recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim)

pickle.dump(user_recall_items_dict, open(save_path + 'usercf_recall_dict.pkl', 'wb'))

if metric_recall:
    # Evaluate recall performance
    metrics_recall(user_recall_items_dict, trn_last_click_df, topk=recall_item_num)

### 7.5 User Embedding Similarity Recall

Although we didn't directly run the UserCF similarity calculation for the full dataset due to its high memory cost, we can still validate the user-based collaborative filtering approach. Below, we'll use the user embeddings generated from the YouTube DNN process to perform a vector search and find the top-k most similar users for each user. Then, we will use the resulting u2u (user-to-user) similarity matrix to perform UserCF recall. The code for this is shown as follows.

In [47]:
# Use the embedding method to get the user-to-user (u2u) similarity matrix
# topk refers to the number of most similar users returned by Faiss for each user
def u2u_embdding_sim(user_emb_dict, save_path, topk):
    user_list = []
    user_emb_list = []
    
    for user_id, user_emb in user_emb_dict.items():
        user_list.append(user_id)
        user_emb_list.append(user_emb)

    user_index_2_rawid_dict = {k: v for k, v in zip(range(len(user_list)), user_list)}

    user_emb_np = np.array(user_emb_list, dtype=np.float32)
    # Build the Faiss index
    user_index = faiss.IndexFlatIP(user_emb_np.shape[1])
    user_index.add(user_emb_np)
    # Perform a similarity search, returning the top-k items and their similarities for each vector in the index
    sim, idx = user_index.search(user_emb_np, topk)  # Returns a list

    # Save the vector retrieval results with a correspondence to the original IDs
    user_sim_dict = defaultdict(dict)
    for target_idx, sim_value_list, rele_idx_list in tqdm(zip(range(len(user_emb_np)), sim, idx)):
        target_raw_id = user_index_2_rawid_dict[target_idx]
        # Start from 1 to remove the user's own ID, so the final similar users obtained are topk-1
        for rele_idx, sim_value in zip(rele_idx_list[1:], sim_value_list[1:]):
            rele_raw_id = user_index_2_rawid_dict[rele_idx]
            user_sim_dict[target_raw_id][rele_raw_id] = user_sim_dict.get(target_raw_id, {}).get(rele_raw_id, 0) + sim_value

    # Save the u2u similarity matrix
    pickle.dump(user_sim_dict, open(save_path + 'youtube_u2u_sim.pkl', 'wb'))
    return user_sim_dict

In [48]:
# Read the user embeddings generated from the YouTubeDNN process, then use Faiss to calculate
# the similarity between users.
# Note: The user embeddings obtained here may not be optimal, as YouTubeDNN trains user embeddings
# using user click sequences. If the sequences are generally short, the results may not be very good.
user_emb_dict = pickle.load(open(save_path + 'user_youtube_emb.pkl', 'rb'))
u2u_sim = u2u_embdding_sim(user_emb_dict, save_path, topk=10)

200000it [00:08, 23078.49it/s]


In [49]:
# Use the recall evaluation function to verify the performance of the current recall method.
if metric_recall:
    trn_hist_click_df, trn_last_click_df = get_hist_and_last_click(all_click_df)
else:
    trn_hist_click_df = all_click_df

user_recall_items_dict = defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)
u2u_sim = pickle.load(open(save_path + 'youtube_u2u_sim.pkl', 'rb'))

sim_user_topk = 50
recall_item_num = 50
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = user_based_recommend(user, user_item_time_dict, u2u_sim, sim_user_topk, \
                                                        recall_item_num, item_topk_click, item_created_time_dict, emb_i2i_sim)

user_multi_recall_dict['youtubednn_usercf_recall'] = user_recall_items_dict
pickle.dump(user_multi_recall_dict['youtubednn_usercf_recall'], open(save_path + 'youtubednn_usercf_recall_dict.pkl', 'wb'))

if metric_recall:
    # Evaluate recall performance
    metrics_recall(user_multi_recall_dict['youtubednn_usercf_recall'], trn_last_click_df, topk=recall_item_num)

100%|████████████████████████████████████████████████████████████████████████| 200000/200000 [02:39<00:00, 1250.07it/s]


topk: 10  | hit_num: 2999  | hit_rate: 0.01499  | user_num: 200000
topk: 20  | hit_num: 7125  | hit_rate: 0.03562  | user_num: 200000
topk: 30  | hit_num: 15088  | hit_rate: 0.07544  | user_num: 200000
topk: 40  | hit_num: 24493  | hit_rate: 0.12247  | user_num: 200000
topk: 50  | hit_num: 33096  | hit_rate: 0.16548  | user_num: 200000


## 8. Cold-start Issue

Cold-start is a classic problem in recommendation systems. It occurs when the system lacks sufficient data to make accurate recommendations. The problem can be broken down into three main categories:

- **Item Cold-Start**: This happens with new articles that have no click history. The challenge is to figure out how to recommend this new content to users. In our scenario, any article not present in the log data can be considered a cold-start article.

- **User Cold-Start**: This affects new users who have just joined the platform and have no interaction history. The system doesn't know what to recommend. In our case, a user in the test set who has only clicked once could be considered a cold-start user.

- **System Cold-Start**: This is a combination of the above two problems. It happens when a brand new platform has no historical data for either users or articles.

#### Our Current Cold-Start Challenge
In our current dataset, we're facing significant cold-start issues: 
1. There are over 300,000 articles in the total library, but only around 30,000 of them have any click data. This means a huge number of articles have no interaction history, which is the definition of article cold-start.
2. We also see signs of user cold-start: nearly 20% of the users in our test set have only one click. Recommending content to these users is difficult because we have very little to go on.

While both of these problems are important, we'll focus primarily on article cold-start and its solutions. The user cold-start problem can be solved by recommending top articles, or passing the user embedding vector to the user tower of a two-tower model (e.g. YoutubeDNN) and retrieve most similar item embedding vectors via ANN (faiss).

#### A Proposed Solution for Article Cold-Start
Our goal isn't just to find articles that don't have click data, but to find articles that a user might actually click. We can treat this as a specialized recall strategy. Here's a possible approach:

1. Embedding-based Recall: First, we can use item embeddings to find a set of articles that are similar to the ones a user has historically clicked on.

2. Rule-based Filtering: From this recalled set, we can apply some simple rules to filter the articles. For example, we could keep only articles that match the user's preferred topics or reading length. We should also prioritize articles that were created around the same time as the user's last click, as this can capture a user's interest in a timely event.

This approach is different from a standard embedding-based recall. Our goal is to find similar articles that have not appeared in the log data. This means we need to recall a larger number of candidates initially, as many will be filtered out by our cold-start rules.

In [50]:
# First, perform ItemCF recall.
trn_hist_click_df = all_click_df

user_recall_items_dict = defaultdict(dict)
user_item_time_dict = get_user_item_time(trn_hist_click_df)
i2i_sim = pickle.load(open(save_path + 'emb_i2i_sim.pkl','rb'))

sim_item_topk = 150
recall_item_num = 100  # Recall a slightly larger number of articles to make subsequent filtering easier.
item_topk_click = get_item_topk_click(trn_hist_click_df, k=50)

for user in tqdm(trn_hist_click_df['user_id'].unique()):
    user_recall_items_dict[user] = item_based_recommend(user, user_item_time_dict, i2i_sim, sim_item_topk,
                                                        recall_item_num, item_topk_click,item_created_time_dict, emb_i2i_sim)
pickle.dump(user_recall_items_dict, open(save_path + 'cold_start_items_raw_dict.pkl', 'wb'))

100%|████████████████████████████████████████████████████████████████████████| 200000/200000 [01:18<00:00, 2549.91it/s]


In [51]:
# Filter articles based on rules
# Retain articles with topics similar to the user's historical browsing topics
# Retain articles with a word count that is not significantly different from the user's historical articles
# Retain articles created on the same day as the user's last click
# Return the final results sorted by similarity score

def get_click_article_ids_set(click_df):
    return set(click_df.click_article_id.values)

def cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, \
                     user_last_item_created_time_dict, item_type_dict, item_words_dict,
                     item_created_time_dict, click_article_ids_set, recall_item_num):
    """
    Recalls articles for a cold-start scenario.
    :param user_recall_items_dict: A dictionary of many articles recalled based on embedding similarity.
                                    Example: {user1: [(item1, item2), ..], }
    :param user_hist_item_typs_dict: A dictionary mapping user IDs to the topics of their clicked articles.
    :param user_hist_item_words_dict: A dictionary mapping user IDs to the word count of their historical articles.
    :param user_last_item_created_time_dict: A dictionary mapping user IDs to the creation time of their last clicked article.
    :param item_type_dict: A dictionary mapping article IDs to their topics.
    :param item_words_dict: A dictionary mapping article IDs to their word counts.
    :param item_created_time_dict: A dictionary mapping article IDs to their creation times.
    :param click_article_ids_set: A set of articles clicked by users (i.e., articles that have appeared in the log).
    :param recall_item_num: The number of articles to recall. This refers to articles that have not appeared in the log data.
    """
    cold_start_user_items_dict = {}
    for user, item_list in tqdm(user_recall_items_dict.items()):
        cold_start_user_items_dict.setdefault(user, [])

        # Get historical article information
        hist_item_type_set = user_hist_item_typs_dict[user]
        hist_mean_words = user_hist_item_words_dict[user]
        hist_last_item_created_time = user_last_item_created_time_dict[user]
        hist_last_item_created_time = datetime.fromtimestamp(hist_last_item_created_time)
        
        for item, score in item_list:
            # Get information for the current recalled article
            curr_item_type = item_type_dict[item]
            curr_item_words = item_words_dict[item]
            curr_item_created_time = item_created_time_dict[item]
            curr_item_created_time = datetime.fromtimestamp(curr_item_created_time)

            # First, the article must not have been clicked by the user. Then, filter based on article topic,
            # word count, and creation time.
            if curr_item_type not in hist_item_type_set or \
                item in click_article_ids_set or \
                abs(curr_item_words - hist_mean_words) > 200 or \
                abs((curr_item_created_time - hist_last_item_created_time).days) > 90:
                continue

            cold_start_user_items_dict[user].append((item, score))      # {user1: [(item1, score1), (item2, score2)..]...}

    # Control the number of cold-start recalled articles
    cold_start_user_items_dict = {k: sorted(v, key=lambda x:x[1], reverse=True)[:recall_item_num] \
                                  for k, v in cold_start_user_items_dict.items()}

    pickle.dump(cold_start_user_items_dict, open(save_path + 'cold_start_user_items_dict.pkl', 'wb'))

    return cold_start_user_items_dict

In [52]:
all_click_df_ = all_click_df.copy()
all_click_df_ = all_click_df_.merge(item_info_df, how='left', on='click_article_id')
user_hist_item_typs_dict, user_hist_item_ids_dict, user_hist_item_words_dict, user_last_item_created_time_dict = get_user_hist_item_info_dict(all_click_df_)
click_article_ids_set = get_click_article_ids_set(all_click_df)

# Note: We are using many rules to filter cold-start articles here,
# so the previous recall stage should have recalled a much larger number of articles.
# Otherwise, it's easy to have no articles left after filtering.
cold_start_user_items_dict = cold_start_items(user_recall_items_dict, user_hist_item_typs_dict, user_hist_item_words_dict, \
                                              user_last_item_created_time_dict, item_type_dict, item_words_dict, \
                                              item_created_time_dict, click_article_ids_set, recall_item_num)

user_multi_recall_dict['cold_start_recall'] = cold_start_user_items_dict

100%|████████████████████████████████████████████████████████████████████████| 200000/200000 [00:26<00:00, 7615.05it/s]


## 9. Multi-Channel Recall Merging


Multi-channel recall merging combines the lists of articles retrieved by all of our recall strategies. Below is a summary of all the recall results:

1. Recall based on the item-to-item similarity calculated by Item-based Collaborative Filtering (itemcf).

2. Recall based on the item-to-item similarity calculated using embeddings.

3. YouTube DNN recall.

4. Recall based on the user-to-user similarity calculated from YouTube DNN embeddings.

5. Recall from the cold-start strategy.

Note:
Some strategies may perform better than others. Therefore, we can manually define different weights for the results from each recall channel to perform a final similarity-based fusion.

In [53]:
def combine_recall_results(user_multi_recall_dict, weight_dict=None, topk=25):
    final_recall_items_dict = {}

    # Normalize the scores for each user for each recall method,
    # to allow for score summation across different recall channels.
    def norm_user_recall_items_sim(sorted_item_list):
        # If there are no articles or only one, return the list directly.
        # This can happen when the number of cold-start recalled articles is too small,
        # and all of them are filtered out. We could implement other strategies here.
        if len(sorted_item_list) < 2:
            return sorted_item_list

        min_sim = sorted_item_list[-1][1]
        max_sim = sorted_item_list[0][1]

        norm_sorted_item_list = []
        for item, score in sorted_item_list:
            if max_sim > 0:
                norm_score = 1.0 * (score - min_sim) / (max_sim - min_sim) if max_sim > min_sim else 1.0
            else:
                norm_score = 0.0
            norm_sorted_item_list.append((item, norm_score))

        return norm_sorted_item_list

    print('Merging multi-channel recall...')
    for method, user_recall_items in tqdm(user_multi_recall_dict.items()):
        print(method + '...')
        # When calculating the final recall results, a weight can be set for each recall method.
        if weight_dict == None:
            recall_method_weight = 1
        else:
            recall_method_weight = weight_dict[method]

        for user_id, sorted_item_list in user_recall_items.items(): # 进行归一化
            user_recall_items[user_id] = norm_user_recall_items_sim(sorted_item_list)

        for user_id, sorted_item_list in user_recall_items.items():
            # print('user_id')
            final_recall_items_dict.setdefault(user_id, {})
            for item, score in sorted_item_list:
                final_recall_items_dict[user_id].setdefault(item, 0)
                final_recall_items_dict[user_id][item] += recall_method_weight * score

    final_recall_items_dict_rank = {}
    # When combining multiple recall channels, we can also control the final number of recalled items.
    for user, recall_item_dict in final_recall_items_dict.items():
        final_recall_items_dict_rank[user] = sorted(recall_item_dict.items(), key=lambda x: x[1], reverse=True)[:topk]

    # Save the final dictionary of merged recall results locally.
    pickle.dump(final_recall_items_dict_rank, open(os.path.join(save_path, 'final_recall_items_dict.pkl'),'wb'))

    return final_recall_items_dict_rank  

In [54]:
# Here, we directly assigned the same weight to all multi-channel recall methods.
# These parameter values can be adjusted based on previous recall evaluations.
weight_dict = {'itemcf_sim_itemcf_recall': 1.0,
               'embedding_sim_item_recall': 1.0,
               'youtubednn_recall': 1.0,
               'youtubednn_usercf_recall': 1.0,
               'cold_start_recall': 1.0}

In [55]:
# After the final merge, N items are recalled for each user for ranking
final_recall_items_dict_rank = combine_recall_results(user_multi_recall_dict, weight_dict, topk=150)

Merging multi-channel recall...


  0%|                                                                                            | 0/5 [00:00<?, ?it/s]

itemcf_sim_itemcf_recall...


 20%|████████████████▊                                                                   | 1/5 [00:20<01:20, 20.08s/it]

embedding_sim_item_recall...


...

youtubednn_usercf_recall...


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:41<00:27, 27.18s/it]

cold_start_recall...


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:07<00:00, 25.49s/it]


MemoryError: 

## 9. Summary
We implemented and evaluated several recall strategies:

1. Item-based Collaborative Filtering with association rules.
2. User-based Collaborative Filtering with association rules.
3. YouTube DNN Recall.
4. Cold-Start Recall with content-based rules.